# vacuum

> Support code for defining the discrete states and discrete actions of a vacuum cleaning robot.

In [ ]:
#| default_exp vacuum

In [ ]:
#| include: false
from fastcore.all import *

In [ ]:
#| export

## State

In [ ]:
#| export
rooms = ["Living Room", "Kitchen", "Office", "Hallway", "Dining Room"]

In [ ]:
rooms

['Living Room', 'Kitchen', 'Office', 'Hallway', 'Dining Room']

## Actions

In [ ]:
#| export
action_space = ["L", "R", "U", "D"]
action_spec = """
    1/0/0/0/0 2/8/0/0/0 1/0/0/0/0 2/0/0/8/0
    8/2/0/0/0 0/1/0/0/0 0/1/0/0/0 0/2/0/0/8
    0/0/1/0/0 0/0/2/8/0 0/0/1/0/0 0/0/1/0/0
    0/0/8/2/0 0/0/0/2/8 8/0/0/2/0 0/0/0/1/0
    0/0/0/8/2 0/0/0/0/1 0/8/0/0/2 0/0/0/0/1
    """

In [ ]:
action_space

['L', 'R', 'U', 'D']

In [ ]:
import gtsam
from gtbook.discrete import Variables
from gtbook.display import pretty

In [ ]:
VARIABLES = Variables()

X = VARIABLES.discrete_series("X", [1, 2, 3], rooms) # states for times 1,2 and 3
A = VARIABLES.discrete_series("A", [1, 2], action_space) # actions for times 1 and 2
motion_model = gtsam.DiscreteConditional(X[2], [X[1], A[1]], action_spec)
pretty(motion_model)


X1,A1,0,1,2,3,4
0,0,1,0,0,0,0
0,1,0.2,0.8,0,0,0
0,2,1,0,0,0,0
0,3,0.2,0,0,0.8,0
1,0,0.8,0.2,0,0,0
1,1,0,1,0,0,0
1,2,0,1,0,0,0
1,3,0,0.2,0,0,0.8
2,0,0,0,1,0,0
2,1,0,0,0.2,0.8,0


## Sensing

In [ ]:
#| export
light_levels = ["dark", "medium", "light"]
sensor_spec = "1/1/8 1/1/8 2/7/1 8/1/1 1/8/1"

In [ ]:
sensor_spec

'1/1/8 1/1/8 2/7/1 8/1/1 1/8/1'